# Install and Defaults

In [1]:
%pip install -e ../bayes-kit
%load_ext autoreload
%autoreload 2

Obtaining file:///Users/gil2rok/school/flatiron/bayes-kit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for bayes-kit (pyproject.toml) ... done
  Created wheel for bayes-kit: filename=bayes_kit-0.0.1-0.editable-py3-none-any.whl size=4942 sha256=ce9f35f7f0fca387d5af8636a9b65a392fdee67f2d4436d81fad3559ce5050f4
  Stored in directory: /private/var/folders/sm/j_xxwpn91bb8wdfzfy3mn5600000gn/T/pip-ephem-wheel-cache-nszjlxok/wheels/ca/dd/8e/acee66b0432b70745a44fbdd89e7ff9ed730279d05a10b63bd
Successfully built bayes-kit
  Attempting uninstall: bayes-kit
    Found existing installation: bayes-kit 0.0.1
    Uninstalling bayes-kit-0.0.1:
      Successfully uninstalled bayes-kit-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from pathlib import Path
from collections import namedtuple

import numpy as np
import bridgestan as bs
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid

from utils import DualAveragingStepSize, compute_hash, my_save
from bayes_kit.drghmc import DrGhmcDiag

# Helper Functions

In [3]:
def get_model(hp):
    stan = os.path.join(hp.bridgestan_dir, 'test_models', hp.model_name, hp.model_name + '.stan')
    data = os.path.join(hp.bridgestan_dir, 'test_models', hp.model_name, hp.model_name + '.data.json')
    print(stan)
    print(data)
    model = bs.StanModel.from_stan_file(stan, data)
    
    class Dim:
        def __init__(self, model):
            self.model = model        
        def dims(self):
            return 1
        def log_density(self, theta):
            return self.model.log_density(theta)
        def log_density_gradient(self, theta):
            return self.model.log_density_gradient(theta)
    model = Dim(model)
    
    return model

In [4]:
from bayes_kit import HMCDiag

def get_init_stepsize(hp):
    model = get_model(hp)
    init_stepsize = hp.first_stepsize
    dual_avg = DualAveragingStepSize(initial_step_size=init_stepsize, nadapt=hp.burn_in)
    for i in range(hp.burn_in):
        hmc = HMCDiag(model=model, steps=10, stepsize=init_stepsize)
        _, _, acceptp = hmc.sample(return_acceptp=True)
        init_stepsize = dual_avg(i=i, p_accept=acceptp)
    return init_stepsize

# Experiments

In [5]:
def experiment(sp, hp):
    model = get_model(hp)
    init_stepsize = sp.init_stepsize # get_init_stepsize(hp)
    stepsize = lambda k: init_stepsize * (sp.adaptivity_factor ** -k)
    steps = 1 if sp.leapfrog_steps == "const_num_steps" else lambda k: int(1 / stepsize(k))
    
    sampler = DrGhmcDiag(
        model=model,
        stepsize=stepsize,
        steps=steps,
        num_proposals=sp.num_proposals,
        probabilistic=sp.probabilistic,
        dampening=sp.dampening,
    )
   
    burned_draws = np.asanyarray([sampler.sample()[0] for _ in range(hp.burn_in)])
    draws = np.asanyarray([sampler.sample()[0] for _ in range(hp.chain_length)])
    my_save(sp, hp, burned_draws, draws)

In [6]:
HyperParamsTuple = namedtuple("hyper_params", ["model_name", "burn_in", "chain_length", "save_dir", "bridgestan_dir", "first_stepsize"])
SamplerParamsTuple = namedtuple("model_params", ["init_stepsize", "adaptivity_factor", "leapfrog_steps", "dampening", "num_proposals", "probabilistic"])

def experiment_runner():
    # although burn_in and chain_length are sampler parameters, 
    # we do not grid search over them and treat them as hyperparameters
    hp = HyperParamsTuple(
        model_name='bernoulli',
        burn_in=100,
        save_dir='res',
        bridgestan_dir='.bridgestan/bridgestan-2.1.1/',
        chain_length=1000,
        first_stepsize=0.1
    )
    
    sampler_param_grid = ParameterGrid(
        {"init_stepsize": [3e-1, 1e-1, 1e-2, 1e-3],
        "adaptivity_factor": [2, 4],
        "leapfrog_steps": ["const_num_steps", "const_trajectory_length"],
        "dampening": [0.1, 0.2, 0.05],
        "num_proposals": [2, 3, 4, 5],
        "probabilistic": [True, False]}
    )

    bs.set_bridgestan_path(hp.bridgestan_dir)
    # TODO: dispatch in parallel on slurm
    for sampler_params in tqdm(sampler_param_grid):
        sp = SamplerParamsTuple(**sampler_params)
        experiment(sp, hp)

In [10]:
experiment_runner()

  0%|          | 0/384 [00:00<?, ?it/s]

.bridgestan/bridgestan-2.1.1/test_models/bernoulli/bernoulli.stan
.bridgestan/bridgestan-2.1.1/test_models/bernoulli/bernoulli.data.json


OSError: dlopen(/Users/gil2rok/school/flatiron/drghmc/.bridgestan/bridgestan-2.1.1/test_models/bernoulli/bernoulli_model.so, 0x0006): Library not loaded: @rpath/libtbb.dylib
  Referenced from: <D69E5D4F-5B7E-31C6-921A-E05964A5BB99> /Users/gil2rok/school/flatiron/drghmc/.bridgestan/bridgestan-2.1.1/test_models/bernoulli/bernoulli_model.so
  Reason: tried: '/Users/gil2rok/school/flatiron/project/.bridgestan/bridgestan-2.1.1/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/gil2rok/school/flatiron/project/.bridgestan/bridgestan-2.1.1/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such file), '/Users/gil2rok/school/flatiron/project/.bridgestan/bridgestan-2.1.1/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/gil2rok/school/flatiron/project/.bridgestan/bridgestan-2.1.1/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such file), '/Users/gil2rok/school/flatiron/project/.bridgestan/bridgestan-2.1.